In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import cv2
import cv2 as cv
plt.rcParams['figure.figsize'] = [15, 10]
import collections, numpy
import glob
import random

def random_color():
    levels = range(32,256,32)
    return tuple(random.choice(levels) for _ in range(3))

## Cart 2

In [ ]:
img_o = cv2.imread('inputs/P3/cart_2.png',0)
plt.imshow(img_o,cmap="gray")

ORGINAL_ITEMS = []
Prices = [40,83,6.5,30,5,50,8,108,6,139,144,36]
Name = ["Tuna-edible","Pril","Soda-drink","MihanMilk-drink","TinyCake-edible",
        "Toothpaste","MihanIceCreamCake-edible","Oil",
        "MihanIceCreamLivani-edible","Shampoo","Tea","Chips-edible"]
paths = glob.glob("./inputs/P3/items/*.png")
paths.sort()
for p in paths:
    tmp = cv2.imread(p,0)
    ORGINAL_ITEMS.append(cv2.blur(tmp,(7,7)))

def imshow(img,ti="",show=True):
    plt.imshow(img,cmap="gray")
    plt.axis("off")
    if ti != "":
        plt.title(ti)
    if show:
        plt.show()

img_o = img_o[50:,:]
img_o = img_o[:1075,:]
img_o = cv2.blur(img_o,(7,7))
imshow(img_o,"Gray Scale of input image")
img = img_o.copy()

In [ ]:
def is_detect(img1,img2,MIN_MATCH_COUNT=10,di_coe=0.5):
    img1 = img1.copy()
    img2 = img2.copy()
    sift = cv.SIFT_create()
    kp1, des1 = sift.detectAndCompute(img1,None)
    kp2, des2 = sift.detectAndCompute(img2,None)
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks = 50)
    flann = cv.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(des1,des2,k=2)

    good = []
    for m,n in matches:
        if m.distance < di_coe*n.distance:
            good.append(m)
            
    if len(good)>MIN_MATCH_COUNT:
        src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
        dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)
        M, mask = cv.findHomography(dst_pts,src_pts, cv.RANSAC,5.0)
        matchesMask = mask.ravel().tolist()
        h,w = img2.shape
        pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
        dst = cv.perspectiveTransform(pts,M)   
        co = random_color()
        draw_params = dict(matchColor = co, # draw matches in green color
                   singlePointColor = None,
                   matchesMask = matchesMask, # draw only inliers
                   flags = 2)
        img3 = cv.drawMatches(img1,kp1,img2,kp2,good,None,**draw_params)
        cv2.fillPoly(img1, pts =[np.int32(dst)], color=(128,128,128))
        return True,img1,img3
    else:
        return False,None,None
   
   
plt.rcParams['figure.figsize'] = [15, 15]

def search_elements(input_img,log=False):
    global Name
    global Prices
    global ORGINAL_ITEMS
    results = np.zeros(shape=(len(Name),),dtype=np.int32)
    input_img = input_img.copy()
    plt_index = 1
    for index in range(len(Name)):
        
        stat,tmp,log_img = is_detect(input_img,ORGINAL_ITEMS[index])
        while stat :
            results[index] += 1
            input_img = tmp
            if log :
                title = Name[index].split("-")[0]
                title += " is detected for #{0} time".format(results[index])
                plt.subplot(3,1,plt_index)
                plt_index += 1
                imshow(log_img,title,False)
                if plt_index == 4 :
                    plt_index = 1
                    plt.tight_layout()
                    plt.show()
                
            stat,tmp,log_img = is_detect(input_img,ORGINAL_ITEMS[index])   
    if log and plt_index != 1 :
        plt.tight_layout()
        plt.show()
    return results 

list_of_cart = search_elements(img,True)

### #f

In [ ]:
for i in range(len(Name)):
    print(Name[i].split("-")[0]+":",list_of_cart[i])
print("==================")
print("Total:",np.sum(list_of_cart)) 

### #g

In [ ]:
for i in range(len(Name)):
    if "chips" in Name[i].lower():
        print(Name[i].split("-")[0]+":",list_of_cart[i])

### #h

In [ ]:
t = 0 
its = 0 
for i in range(len(Name)):
    if "icecream" in Name[i].lower():
        print("--------")
        print(Name[i].split("-")[0]+" / Single price:{0}KT / Count:{1} / TotalCost:{2}KT".format(Prices[i],list_of_cart[i],list_of_cart[i]*Prices[i]))
        t+= list_of_cart[i]*Prices[i]
        its += list_of_cart[i]

print("==================")
print("Total Items:",its,"- Total Price:{0}KT".format(t)) 


### #i

In [ ]:
t = 0 
its = 0 
for i in range(len(Name)):
    if "edible" in Name[i].lower():
        print("--------")
        print(Name[i].split("-")[0]+" / Single price:{0}KT / Count:{1} / TotalCost:{2}KT".format(Prices[i],list_of_cart[i],list_of_cart[i]*Prices[i]))
        t+= list_of_cart[i]*Prices[i]
        its += list_of_cart[i]

print("==================")
print("Total Items:",its,"- Total Price:{0}KT".format(t)) 

### #j

In [ ]:
t = 0 
for i in range(len(Name)):
    print("--------")
    print(Name[i].split("-")[0]+" / Single price:{0}KT / Count:{1} / TotalCost:{2}KT".format(Prices[i],list_of_cart[i],list_of_cart[i]*Prices[i]))
    t+= list_of_cart[i]*Prices[i]
    
print("==================")
print("Total Items:",np.sum(list_of_cart),"- Total Price:{0}KT".format(t)) 

## Repeat Cart 1

In [ ]:
img_o = cv2.imread('inputs/P3/cart_1.png',0)
plt.imshow(img_o,cmap="gray")

ORGINAL_ITEMS = []
Prices = [40,83,6.5,30,5,50,8,108,6,139,144,36]
Name = ["Tuna-edible","Pril","Soda-drink","MihanMilk-drink","TinyCake-edible",
        "Toothpaste","MihanIceCreamCake-edible","Oil",
        "MihanIceCreamLivani-edible","Shampoo","Tea","Chips-edible"]
paths = glob.glob("./inputs/P3/items/*.png")
paths.sort()
for p in paths:
    tmp = cv2.imread(p,0)
    ORGINAL_ITEMS.append(cv2.blur(tmp,(7,7)))

def imshow(img,ti="",show=True):
    plt.imshow(img,cmap="gray")
    plt.axis("off")
    if ti != "":
        plt.title(ti)
    if show:
        plt.show()

img_o = img_o[50:,:]
img_o = img_o[:1075,:]
img_o = cv2.blur(img_o,(7,7))
imshow(img_o,"Gray Scale of input image")
img = img_o.copy()

list_of_cart = search_elements(img,True)

### #a

In [ ]:
for i in range(len(Name)):
    print(Name[i].split("-")[0]+":",list_of_cart[i])
print("==================")
print("Total:",np.sum(list_of_cart)) 

### #b

In [ ]:
t = 0 
for i in range(len(Name)):
    if "tiny" in Name[i].lower():
        print(Name[i].split("-")[0]+":",list_of_cart[i])
        t += list_of_cart[i]

print("==================")
print("Total Tinies:",t) 

### #c

In [ ]:
t = 0 
for i in range(len(Name)):
    if "drink" in Name[i].lower():
        print(Name[i].split("-")[0]+", Count:",list_of_cart[i], ", Single price:",Prices[i], ", Total price:",Prices[i]*list_of_cart[i])
        t += list_of_cart[i]*Prices[i]

print("==================")
print("Total Drinks: {0}KT".format(t)) 

### #d

In [ ]:
t = 0 
for i in range(len(Name)):
    if "mihan" in Name[i].lower():
        print(Name[i].split("-")[0]+":",list_of_cart[i])
        t += list_of_cart[i]

print("==================")
print("Total by Mihan:",t) 

In [ ]:
t = 0 
for i in range(len(Name)):
    print("--------")
    print(Name[i].split("-")[0]+" / Single price:{0}KT / Count:{1} / TotalCost:{2}KT".format(Prices[i],list_of_cart[i],list_of_cart[i]*Prices[i]))
    t+= list_of_cart[i]*Prices[i]
    
print("==================")
print("Total Items:",np.sum(list_of_cart),"- Total Price:{0}KT".format(t)) 